#The Spiking Neuron Model - Coding Challenge Problems (Part 1)


#Integrate-and-Fire Spiking Neuron Model
This interactive document is meant to be followed as the reader makes their way through chapter: *The Spiking Neuron Model*.  Each model presented in the chapter will have a section consisting of a step-by-step walkthrough of a simple Python implementation.  This is followed by an interface to run simulations with different parameter values to answer the Coding Challenge Problems.

For each model covered in the chapter, there is a section called **Simulations for Coding Challenge Problems.**  This is where you will find user-interface components such as value sliders for various parameters.  Use these controls to answer the questions from the text, and feel free to explore how the values affect the results.


**Content creators**: Maxwell E. Bohling, Dr. Lawrence Udeigwe

## How This Works

This Jupyter Notebook has both Content cells and Code cells.  As you make your way through the tutorial, you MUST make sure to run each code cell as you come to them.  Each code cell has a *Play* button next to it which will execute the code. Some code may be hidden (generally because it is more complex code not completely necessary to understand for the purposes of the chapter Coding Challenge Problems.)

**IMPORTANT**: You are currently viewing a copy of the original notebook.  You will find that you can edit the content of any cell.  If you accidently change a cell, such as a line of code and/or run into errors as you try to run subsequent blocks, simply refresh the page, OR go to the *Runtime* Menu and select *Restart runtime*.  It is also suggested that you go to the *Edit* menu and select *Clear all outputs.*  This will always allow you to revert the notebook to the original version (though you will have to run each code block again.)



 Execute the code block. **Initialize Setup**

In [ ]:
#@title Initialize Setup
#@markdown Import python libraries, and define helper functions (No need to understand this code, simply make sure you run this first.)
import sys
import functools as ft
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy as sc

def printError():
  message = 'Something went wrong!\n\n'  
  message = message + 'Check for the following:\n\n'
  message = message + '\t1.  Variables are set to the correct values.  Make sure to replace any instances of \'None\'.\n'
  message = message + '\t2.  All previous code blocks have been run the order they appear and output a success message.\n'
  message = message + '\t3.  No other code has been altered.\n\n'
  message = message + 'then try running the code block again.\n'
  print(message)

def printSuccess():
  message = 'Success!  Move on to the next section.'
  print(message)

def checkVoltageParameters(Vrest, Vth, Vreset, Vspike):
  # Check: IAF-Voltage-Parameters
  print('Checking Voltage Parameters...\n')
  try:
      check_Vrest = Vrest
      check_Vth = Vth
      check_Vreset = Vreset
      check_Vspike = Vspike
  except:
    printError()
  else:
    vals = [Vrest, Vth, Vreset, Vspike]
    correct_vals = [-70, -54, -80, 0]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  True):  
      printSuccess()
    else:
      printError()
 
def checkNeuronProperties(Rm, Cm, tau_m):
  # Check: IAF-Neuron-Properties
  print('Checking Neuron Properties...\n')
  try:
      check_Rm = Rm
      check_Cm = Cm
      check_tau_m = tau_m
  except:
    printError()
  else:
    vals = [Rm, Cm, tau_m]
    correct_vals = [10, 1, 10]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  True):  
      printSuccess()
    else:
      printError()

def checkSimulationSetup(Vrest, Vinitial, EL, t0, dt, t_final, time, Ie, t_pulse, start_current, end_current):
  # Check: IAF-Simulation-Setup
  print('Checking Simulation Setup...\n')
  try:
    check_Vrest = Vrest
    check_Vinitial = Vinitial
    check_EL = EL
    check_t0 = t0
    check_dt = dt
    check_t_final = t_final
    check_time = time
    check_Ie = Ie
    check_t_pulse = t_pulse
    check_start_current = start_current
    check_end_current = end_current
  except:
    printError()
  else:
    vals = [Vrest, Vinitial, EL, t0, dt, t_final, Ie, t_pulse, start_current, end_current]
    correct_vals = [-70, -70, -70, 0, 1, 500, 1.75, 300, 100, 400]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  True):  
      if len(time) == 500 and time[0] == 0 and time[-1] == 499:
        printSuccess()
      else:
        printError()
    else:
      printError()

def autoSetValues():
  Vrest = -70
  Vth = -54 
  Vreset = -80
  Vspike = 0
  Vinitial = Vrest
  EL = Vrest
  t0 = 0
  dt = 1
  t_final = 500
  time = range(t0, t_final, dt)
  t_pulse = 300 
  start_current = np.absolute(t_final-t_pulse)/2 
  end_current = start_current+t_pulse 
  V = [0] * len(time)
  V[0] = Vrest

try:
  check_sys = sys
except:
  printError()
else:
  modulename = 'functools'
  if modulename not in sys.modules:
    printError()
  else:
    printSuccess()

## Walkthrough
### Membrane Equation
Recall that the basic equation is expressed as follows:

$ \displaystyle \tau_{m}\frac{dV}{dt} = E_{L} - V(t) + R_{m}I_{e} $




### Voltage Parameters

Start by defining the Voltage-Parameters that determine the resting, threshold, spike, and reset values:

*   $ V_{rest} = -70\;mV $
*   $ V_{th} = -54\;mV $
*   $ V_{reset} = -80\;mV $
*   $ V_{spike} = 0\;mV $







In [ ]:
# Voltage Parameters - Units mV (1 mV = 1e-3 Volts)
Vrest = -70
Vth = -54
Vreset = -80
Vspike = 0

checkVoltageParameters(Vrest,Vth,Vreset,Vspike)

### Neuron Properties
We now need to define the values for the Membrane Resistance $ R_{m} $, the Membrane Capacitance, $ C_{m} $, and calculate the Membrane Time Constant.  Use the following values:

*    $ R_{m} = 10\;M\Omega $
*    $ C_{m} = 1\;nF $
*    $ \tau_{m} = R_{m}C_{m} $



In [ ]:
try:
  Rm = 10
  Cm = 1
# Membrane Time Constant - Units ms (1 ms = 1e-3 Seconds)
  tau_m = Rm*Cm
except:
  printError()
else:
  checkNeuronProperties(Rm,Cm,tau_m)

### Simulation Setup
We want to run a simulation of the integrate-and-fire model neuron in order to observe the evolution of the membrane potential $V$ overtime.  We begin with the neuron at resting conditions: $V = V_{rest} = -70mV$.

Recall that the reversal potential $E_{L} = V_{rest}$.  Since we have defined $V_{rest}$ above, we can set the following parameters.

In [ ]:
try:
  # Initial Voltage
  Vinitial = Vrest

  # Leakage Conductance Reversal Potential
  EL = Vrest
except:
  printError()
else:
  printSuccess()

We will be running a 500 ms simulation.  The following lines of code setup a time span for the simulation.  This is simply a matter of defining the start time $t_{0} = 0$ and the total length (in ms) of the simulation: $t_{final} = 500$.  

Throughout the simulation, we calculate the membrane potential $V$ at each *time-step*.  The time-step is the change in time for each iteration of the simulation, for example if $t_{0} = 0$, the next computation of $V$ is performed at $t_{0} + dt$.  

Thus, by setting $dt = 1$ (in ms), the simulation will compute $V$ at time $t = 1, 2, \ldots, t_{final}$.  For $dt = 1$, we compute $V$ at every 1 ms until the end of the simulation is reached.

In [ ]:
try:
  # Simulation Time Span (0 to 500ms, dt = 1ms)
  t0 = 0
  dt = 1
  t_final = 500

  time = range(t0, t_final, dt)
except:
  printError()
else:
  if len(time) == 500 and time[0] == 0 and time[-1] == 499:
    printSuccess()
  else:
    printError()

We must define one last parameter, and that is the injected current $I_{e}$.  There are two aspects of this parameter that must be defined: the strength of the current measured in nanoamperes (nA), and the time values at which the current is switched on and off.

In [ ]:
# Input Current: Ie - Units nA (1 nA = 10-9 Amperes)
Ie = 1.75

try:
  check_Ie = Ie
except:
  printError()
else:
  if Ie == 1.75:
    printSuccess()
  else:
    printError()

Recall that we have set up the simulation to run for 500ms.  We want to begin applying the injected current $I_{e}$ at 100ms, and switch it off at 400 ms.

To do this we define the variables:
*    $ t_{pulse} = 300\ ms $ 
*    $ start_{current} = \displaystyle \frac{\left|t_{final}-t_{pulse}\right|}{2} = \frac{\left|500-300\right|}{2} = 100\ ms$ 
*    $ end_{current} = start_{current} + t_{pulse} = 100 + 300 = 400\ ms $

In [ ]:
try:
  # Current Pulse Time Span - Units ms (1 ms = 1e-3 Seconds)
  t_pulse = 300 

  # Time at which the current is applied
  start_current = np.absolute(t_final-t_pulse)/2 

  # Time at which the current is switched off
  end_current = start_current+t_pulse 
except:
  printError()
else:
  checkSimulationSetup(Vrest, Vinitial, EL, t0, dt, t_final, time, Ie, t_pulse, start_current, end_current)

### Computing and Storing $\frac{dV}{dt}$

We are about ready to finish the code implementation for simulating an integrate-and-fire model neuron.

We only need some way to store the values of the membrane potential $V$ at each time step.  To do this, we simply create an empty list $V(t)$ with a length equal to the number of time-steps of our simulation.  We then set $V(t_{0}) = V(0) = V_{rest}$ as we begin our simulation at resting conditions.

In [ ]:
try:
  # Create a list V(t) to store the value of V at each time-step dt
  V = [0] * len(time)

  # Set the initial value at time t = t0 to the initial value Vinitial
  V[0] =  Vinitial
except:
  printError()
else:
  printSuccess()

Finally, we run our simulation according to the following *psuedocode*

---

*for each time-step from $t = t_{0}$ to $t = t_{final}$*
> *If the current time $t < start_{current}\ $ or $\ t > end_{current}$*
>> $I_{e} = 0$

> *otherwise*
>> $I_{e} = 1.75\ nA$

> *Use Euler's Method of Numerical Integration to determine the change in membrane potential $\ dV$ as: $\ \displaystyle dV = \frac{dt}{\tau_{m}}(E_{L} - V(t) + R_{m}I_{e})$*

> *Update $V(t+1) = V(t) + dV$*

> *If $\ V(t+1) \geq V_{th} $* 
>> $V(t) = V_{spike}$

>> $V(t+1) = V_{reset}$

*end*

---

This translates to the following Python code.

In [ ]:
try:
  # For each timestep we compute V and store the value
  for t in time[0:-2]:

      # If time t < 100 ms, we do not turn the Injected Current on yet.
      # If time t > 400ms, we turn the Injected Current off.
      if t < start_current or t > end_current:
          ie = 0
      # Otherwise, we switch on the Injected Current 
      else:
          ie = Ie

      # Using Euler's Method for Numerical Integration (See Chapter Text)
      # we compute the change in voltage dV as follows (using the model equation)
      dVt = (dt/tau_m)*(EL - V[t] + Rm*ie)

      # Store this new value into our list
      V[t+1] = V[t] + dVt

      # If the voltage value we computed crosses the threshold value Vth, a spike occurs.
      if V[t+1] >= Vth:
          # In the event of a spike, the membrane potential spikes up to Vspike mV
          V[t] = Vspike
          # Followed by immediate hyperpolarization to its Vreset value.
          V[t+1] = Vreset
except:
  printError()
else:
  printSuccess()

### Visualizing Results

Now we have values of $V$ for each time-step of the simulation, we can visualize the results by using Python to plot the data.  This makes use of another widely used library **matplotlib**.

In [ ]:
try:
  check_plt = plt
except:
  printError()
else:
  # Plot the membrane potential
  plt.plot(time[0:-2], V[0:-2])
  plt.title('Integrate-and-Fire Model Neuron: $V$ against Time')
  plt.xlabel('Time (ms)')
  plt.ylabel('$V(t)$')
  plt.hlines(-54,time[0],time[-2],colors='r',linestyles='dashed')

  # Display the plot
  plt.show()       

## Full Integrate-and-Fire Model Neuron Code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

# Voltage Paramaters - Units mV (1 mV = 1e-3 Volts)
Vrest = -70
Vth = -54 
Vreset = -80
Vspike = 0

# Total Membrane Resistance - Units M_Ohm (1 M_Ohm = 1e6 Ohms)
Rm = 10

# Total Membrane Capacitance - Units 
Cm = 1

# Membrane Time Constant - Units ms (1 ms = 1e-3 Seconds)
tau_m = Rm*Cm

# Initial V
Vinitial = Vrest
EL = Vrest

# Simulation Time Span (0 to 500ms, dt = 1ms)
t0 = 0
dt = 1
t_final = 500

# Create the list of time step values
time = range(t0, t_final, dt)

# Current Pulse Time Span - Units ms (1 ms = 1e-3 Seconds)
t_pulse = 300 

# Time at which the current is applied
start_current = np.absolute(t_final-t_pulse)/2 

# Time at which the current is switched off
end_current = start_current+t_pulse 

# Input Current: Ie - Units nA (1 nA = 10-9 Amperes)
# Using 1.75 for default
Ie = 1.75

# Create a list V(t) to store the value of V at each time-step dt
V = [0] * len(time)

# Set the initial value at time t = t0 to the resting potential
V[0] = Vrest

for t in time[0:-2]:

  if t < start_current or t > end_current:
      ie = 0
  else:
      ie = Ie

  # General Passive Integrate-and-Fire Model
  dV = (1/tau_m)*(EL - V[t] + Rm*ie)*dt

  V[t+1] = V[t] + dV

  # If V crosses our threshold value Vth
  # 1. Store the Vspike value at time t
  # 2. Set V back to Vreset
  if V[t+1] >= Vth:
      V[t] = Vspike   # 1
      V[t+1] = Vreset # 2

# Plot the membrane potential
plt.plot(time[0:-2], V[0:-2])
plt.title('Integrate-and-Fire Model Neuron: $V$ against Time')
plt.xlabel('Time (ms)')
plt.ylabel('$V(t)$')
plt.hlines(-54,time[0],time[-2],colors='r',linestyles='dashed')

# Display the plot
plt.show()   

## Simulations for Coding Challenge Problems

In [ ]:
#@title Run Simulations
#@markdown Execute the code block and use the sliders to set values in order to answer the Coding Challenge Problems in the chapter text.

autoSetValues()

def compute_iaf_neuron(Ie,Rm,tau_m):
  for t in time[0:-2]:
      if t < start_current or t > end_current:
          ie = 0
      else:
          ie = Ie
      dVt = (dt/tau_m)*(EL - V[t] + Rm*ie)
      V[t+1] = V[t] + dVt
      if V[t+1] >= Vth:
          V[t] = Vspike
          V[t+1] = Vreset
  return V

def plot_iaf_neuron(V, time):
  plt.plot(time[0:-2], V[0:-2])
  plt.title('Integrate-and-Fire Model Neuron: $V$ against Time')
  plt.xlabel('Time (ms)')
  plt.ylabel('$V(t)$')
  plt.hlines(-54,time[0],time[-2],colors='r',linestyles='dashed')
  plt.show()   

@widgets.interact(
  Ie=widgets.FloatSlider(1.75, min=1.50, max=2.50, step=0.01),
  Rm=widgets.IntSlider(10, min=5, max=20, step=1)
)
def simulate_iaf_neuron(Ie=1.75, Rm=10):
    Cm = 1;
    tau_m = Rm*Cm
    V = compute_iaf_neuron(Ie,Rm,tau_m)
    plot_iaf_neuron(V,time)

# Appendix: Importing Libraries

Here, we introduce a feature of Python necessary for many applications: *importing libraries*, sometimes called *packages* or *modules*.

Consider a scenario where you have a small dataset of numerical values:

In [ ]:
# Finding the maximum value

# We use this variable to keep track of the largest value we come across
max_value = 0

# A for-loop: for each number in our data set data
for num in data:
  # If this number is larger than the current max_value
  if num > max_value:
    # Then set max_value to this value
    max_value = num

# Print the result of our algorithm to find the maximum value.
print('Maximum Value: %i' % max_value)

By running the previous code, it should have found the maximum value and the output from the print function is:

`Maximum Value: 99`

Now you are tasked with finding the average value in the data set.  We find this by adding the values of the data set and dividing by the number of values.

The following code is another simple algorithm for computing the average value of a given data set of numerical values:

In [ ]:
# Finding the average value

# Keep track of the total sum of the data set.
sum_of_data = 0

# Let N be the total number of data values
N = 0

# A for-loop: for each number in our data set data
for num in data:
  # Update our variable: sum_of_data by adding each value
  # Note that we do not want to overwrite the value of sum_of_data, but rather 
  # add to the current value
  sum_of_data = sum_of_data + num

  # Update our variable: N by adding 1 for each number we encounter
  N = N + 1

# Finally, compute the average value
average = sum_of_data/N;

# Print the result of our algorithm to find the average value to 2 decimals.
print('Average Value: %.2f' % average)

By running the previous code, it should have found the average value and the output from the print function is:

`Average Value: 68.46`

Now, how does this relate to the concept of **importing libraries**?  Think about the previous algorithms and what they do.  There are innumerable applications in which finding the maximum or average values are necessary.

Do we really need to write these algorithms every single time we need to find the average or maximum values of a data set?

Fortunately, the answer is no!  This is a scenario where we can import a *library* or *package*.  A library is a code base that comes with pre-packaged algorithms ready for you to use.  In other words, another Python programmer has already written the code above, and they are nice enough to allow anyone to use their code.

Let us see how this works by finding the maximum and average values of our data set using a library called **numpy**:


In [ ]:
# Finding the maximum and average values of our small data set

# We import the numpy library, and name the import 'np' 
# np is conventional, however you can call it anything you like
# i.e 'import numpy as abcdefg'
import numpy as np

# First, we find the maximum value and print the result:
max_value = np.max(data)
print('Maximum Value: %i' % max_value)

# Next, we find the average value and print the result:
average = np.mean(data)
print('Average Value: %.2f' % average)

Much easier right?  Note that we compute and output the results using the **numpy** library which already has the algorithms *max* and *mean* to do the job of our previous code in a single line!

These are just some basic examples, there are over 200,000 of these libraries providing access to algorithms helping us to avoid writing unnecessary code.

Visit https://pypi.org/ and look around for libraries and packages ready to be imported into your code!